<table align="left">
    <tr>
        <td style="vertical-align: middle; padding-left: 0px; padding-right: 0px;">
            <a href="https://creativecommons.org/licenses/by/4.0/">
                <img src="https://licensebuttons.net/l/by/4.0/80x15.png" />
            </a>
        </td>
        <td style="vertical-align: middle; padding-left: 5px; padding-right: 0px;">
            <a href="https://opensource.org/licenses/MIT">
                <img src="https://img.shields.io/badge/License-MIT-green.svg" />
            </a>
        </td>
        <td style="vertical-align: middle; padding-left: 15px;">
            &copy; Guillaume Rongier
        </td>
    </tr>
</table>

# Comparison with Landlab's components

This notebook builds upon a simple example to compare the behavior of StratigraPy's components together and with those of Landlab, as a very rough validation of StratigraPy.

### Imports

Let's first import all the required packages and components:

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt

from landlab.components import FlowDirectorSteepest, FlowAccumulator
from landlab.components import LinearDiffuser, AreaSlopeTransporter, SpaceLargeScaleEroder

from stratigrapy import RasterModelGrid
from stratigrapy.components import GravityDrivenDiffuser, GravityDrivenRouter
from stratigrapy.components import WaterDrivenDiffuser, WaterDrivenRouter, FluxDrivenRouter

## 1. Linear diffusion model

We'll use the same simple setting throughout the notebook: a triangular hill being eroded. Let's start by defining the simulation time and time step: 

In [ ]:
timestep = 100.
runtime = 500000.
n_iterations = int(runtime/timestep)
save_step = 1000

We keep the time step small because, contrary to Landlab, StratigraPy's components don't have an adaptive scheme based on the Courant–Friedrichs–Lewy condition. During the runs, we'll save the topography every `save_step` to compare the evolution of the hill with different models. Let's start with a simple linear diffusion from Landlab. First, we define the grid:

In [ ]:
grid = RasterModelGrid((3, 49), xy_spacing=(500., 500.))
grid.set_closed_boundaries_at_grid_edges(False, True, False, True)

Then the initial triangular topography:

In [ ]:
elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation[grid.x_of_node < 12000.] = 1000.*(grid.x_of_node[grid.x_of_node < 12000.])/12000.
elevation[grid.x_of_node >= 12000.] = 1000. - 1000.*(grid.x_of_node[grid.x_of_node >= 12000.] - 12000.)/12000.

Let's define the diffusion component:

In [ ]:
ld = LinearDiffuser(grid, linear_diffusivity=2e2)

Run the simulation while saving the topography at specific iterations:

In [ ]:
elevations_ld = np.empty((grid.shape[1], n_iterations//save_step))
for i in tqdm(range(n_iterations)):
    if i%save_step == 0:
        elevations_ld[:, i//save_step] = elevation.reshape(grid.shape)[1]
    ld.run_one_step(timestep)

And visualize our base case:

In [ ]:
fig, ax = plt.subplots()

# Plot the results from Landlab
cmap = mpl.colormaps['Wistia']
colors = cmap(np.linspace(0, 1, elevations_ld.shape[1]))[:, np.newaxis]
for i in range(elevations_ld.shape[1]):
    ax.scatter(grid.x_of_node.reshape(grid.shape)[1], elevations_ld[:, i],
               c=colors[i], s=20, ec='k', lw=0.5)

ax.set(xlabel='x (m)', ylabel='z (m)');

Now let's repeat the same operations but with StratigraPy's components. The grid and initial topography remains the same:

In [ ]:
grid = RasterModelGrid((3, 49),
                       xy_spacing=(500., 500.),
                       number_of_classes=1,
                       initial_allocation=1,
                       number_of_top_layers=0)
grid.set_closed_boundaries_at_grid_edges(False, True, False, True)

In [ ]:
elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation[grid.x_of_node < 12000.] = 1000.*(grid.x_of_node[grid.x_of_node < 12000.])/12000.
elevation[grid.x_of_node >= 12000.] = 1000. - 1000.*(grid.x_of_node[grid.x_of_node >= 12000.] - 12000.)/12000.

StratigraPy's components operate in the continental and marine domain, so we need to define a bathymetry, here equal to 0 m because our setting is fully continental:

In [ ]:
bathymetry = grid.add_zeros('bathymetric__depth', at='node', clobber=True)

Then we can setup StratigraPy's linear diffusion model, without limit on erosion to mimic Landlab's component:

In [ ]:
gdd = GravityDrivenDiffuser(grid,
                            diffusivity_cont=2e2,
                            max_erosion_rate_sed=np.inf,
                            max_erosion_rate_br=1e8)

Now we run the simulation just like before: 

In [ ]:
elevations_gdd = np.empty((grid.shape[1], n_iterations//save_step))
for i in tqdm(range(n_iterations)):
    if i%save_step == 0:
        elevations_gdd[:, i//save_step] = elevation.reshape(grid.shape)[1]
    gdd.run_one_step(timestep, update=True)

And we can compare the result with Landlab's:

In [ ]:
fig, ax = plt.subplots()

# Plot the results from Landlab
cmap = mpl.colormaps['Wistia']
colors = cmap(np.linspace(0, 1, elevations_ld.shape[1]))[:, np.newaxis]
for i in range(elevations_ld.shape[1]):
    ax.scatter(grid.x_of_node.reshape(grid.shape)[1], elevations_ld[:, i],
               c=colors[i], s=20, ec='k', lw=0.5, label='Landlab')
# Plot the results from StratigraPy
ax.plot(grid.x_of_node.reshape(grid.shape)[1], elevations_gdd,
        c='k', lw=0.5, label='GravityDrivenDiffuser')

handles, labels = ax.get_legend_handles_labels()
legend = dict(zip(labels, handles))
ax.legend(legend.values(), legend.keys())
ax.set(xlabel='x (m)', ylabel='z (m)');

Both Landlab's *LinearDiffuser* and StratigraPy's *GravityDrivenDiffuser* are based on a time-explicit finite-volume scheme. Let's compare with StratigraPy's *GravityDrivenRouter*, which uses a routing scheme to simulate sediment erosion and deposition. The code is the same as before, except that we need to multiply the diffusivity by the face width to make the two approaches equivalent (here we're using a regular structured grid so all the cells have the same face width, which is equal to the spacing):

In [ ]:
grid = RasterModelGrid((3, 49),
                       xy_spacing=(500., 500.),
                       number_of_classes=1,
                       initial_allocation=1,
                       number_of_top_layers=0)
grid.set_closed_boundaries_at_grid_edges(False, True, False, True)

elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation[grid.x_of_node < 12000.] = 1000.*(grid.x_of_node[grid.x_of_node < 12000.])/12000.
elevation[grid.x_of_node >= 12000.] = 1000. - 1000.*(grid.x_of_node[grid.x_of_node >= 12000.] - 12000.)/12000.

bathymetry = grid.add_zeros('bathymetric__depth', at='node', clobber=True)

gdr = GravityDrivenRouter(grid,
                          diffusivity_cont=2e2/grid.spacing[0],
                          max_erosion_rate_sed=np.inf,
                          max_erosion_rate_br=1e8)

elevations_gdr = np.empty((grid.shape[1], n_iterations//save_step))
for i in tqdm(range(n_iterations)):
    if i%save_step == 0:
        elevations_gdr[:, i//save_step] = elevation.reshape(grid.shape)[1]
    gdr.run_one_step(timestep, update=True)

And we can compare the result with Landlab's:

In [ ]:
fig, ax = plt.subplots()

# Plot the results from Landlab
cmap = mpl.colormaps['Wistia']
colors = cmap(np.linspace(0, 1, elevations_ld.shape[1]))[:, np.newaxis]
for i in range(elevations_ld.shape[1]):
    ax.scatter(grid.x_of_node.reshape(grid.shape)[1], elevations_ld[:, i],
               c=colors[i], s=20, ec='k', lw=0.5, label='Landlab')
# Plot the results from StratigraPy
ax.plot(grid.x_of_node.reshape(grid.shape)[1], elevations_gdr,
        c='k', lw=0.5, label='GravityDrivenRouter')

handles, labels = ax.get_legend_handles_labels()
legend = dict(zip(labels, handles))
ax.legend(legend.values(), legend.keys())
ax.set(xlabel='x (m)', ylabel='z (m)');

We cannot check the multi-sediment-class case, but we can at least check that two sediment classes with the same parameter values lead to the same result:

In [ ]:
grid = RasterModelGrid((3, 49),
                       xy_spacing=(500., 500.),
                       number_of_classes=2,
                       initial_allocation=1,
                       number_of_top_layers=0)
grid.set_closed_boundaries_at_grid_edges(False, True, False, True)

elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation[grid.x_of_node < 12000.] = 1000.*(grid.x_of_node[grid.x_of_node < 12000.])/12000.
elevation[grid.x_of_node >= 12000.] = 1000. - 1000.*(grid.x_of_node[grid.x_of_node >= 12000.] - 12000.)/12000.

bathymetry = grid.add_zeros('bathymetric__depth', at='node', clobber=True)

gdr = GravityDrivenRouter(grid,
                          diffusivity_cont=[2e2/grid.spacing[0], 2e2/grid.spacing[0]],
                          max_erosion_rate_sed=np.inf,
                          bedrock_composition=[0.5, 0.5],
                          max_erosion_rate_br=1e8)

elevations_gdr = np.empty((grid.shape[1], n_iterations//save_step))
for i in tqdm(range(n_iterations)):
    if i%save_step == 0:
        elevations_gdr[:, i//save_step] = elevation.reshape(grid.shape)[1]
    gdr.run_one_step(timestep, update=True)

And indeed it does:

In [ ]:
fig, ax = plt.subplots()

# Plot the results from Landlab
cmap = mpl.colormaps['Wistia']
colors = cmap(np.linspace(0, 1, elevations_ld.shape[1]))[:, np.newaxis]
for i in range(elevations_ld.shape[1]):
    ax.scatter(grid.x_of_node.reshape(grid.shape)[1], elevations_ld[:, i],
               c=colors[i], s=20, ec='k', lw=0.5, label='Landlab')
# Plot the results from StratigraPy
ax.plot(grid.x_of_node.reshape(grid.shape)[1], elevations_gdr,
        c='k', lw=0.5, label='GravityDrivenRouter')

handles, labels = ax.get_legend_handles_labels()
legend = dict(zip(labels, handles))
ax.legend(legend.values(), legend.keys())
ax.set(xlabel='x (m)', ylabel='z (m)');

## 2. Non-linear diffusion model

Landlab's non-linear component implements a slightly different method, so instead we will compare the two components from StratigraPy together, to check that the two solving schemes lead to similar outputs. Let's start with *GravityDrivenDiffuser*, which uses the finite-volume approach and will be our base case:

In [ ]:
grid = RasterModelGrid((3, 49),
                       xy_spacing=(500., 500.),
                       number_of_classes=1,
                       initial_allocation=1,
                       number_of_top_layers=0)
grid.set_closed_boundaries_at_grid_edges(False, True, False, True)

elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation[grid.x_of_node < 12000.] = 1000.*(grid.x_of_node[grid.x_of_node < 12000.])/12000.
elevation[grid.x_of_node >= 12000.] = 1000. - 1000.*(grid.x_of_node[grid.x_of_node >= 12000.] - 12000.)/12000.

bathymetry = grid.add_zeros('bathymetric__depth', at='node', clobber=True)

gdd = GravityDrivenDiffuser(grid,
                            diffusivity_cont=2e2,
                            critical_angle_cont=10.,
                            critical_angle_mar=10.,
                            max_erosion_rate_sed=np.inf,
                            max_erosion_rate_br=1e8)

elevations_gdd = np.empty((grid.shape[1], n_iterations//save_step))
for i in tqdm(range(n_iterations)):
    if i%save_step == 0:
        elevations_gdd[:, i//save_step] = elevation.reshape(grid.shape)[1]
    gdd.run_one_step(timestep, update=True)

In [ ]:
fig, ax = plt.subplots()

# Plot the results from GravityDrivenDiffuser
cmap = mpl.colormaps['Wistia']
colors = cmap(np.linspace(0, 1, elevations_gdd.shape[1]))[:, np.newaxis]
for i in range(elevations_gdd.shape[1]):
    ax.scatter(grid.x_of_node.reshape(grid.shape)[1], elevations_gdd[:, i],
               c=colors[i], s=20, ec='k', lw=0.5)

ax.set(xlabel='x (m)', ylabel='z (m)');

Now let's setup and run a *GravityDrivenRouter*, which uses the routing scheme:

In [ ]:
grid = RasterModelGrid((3, 49),
                       xy_spacing=(500., 500.),
                       number_of_classes=1,
                       initial_allocation=1,
                       number_of_top_layers=0)
grid.set_closed_boundaries_at_grid_edges(False, True, False, True)

elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation[grid.x_of_node < 12000.] = 1000.*(grid.x_of_node[grid.x_of_node < 12000.])/12000.
elevation[grid.x_of_node >= 12000.] = 1000. - 1000.*(grid.x_of_node[grid.x_of_node >= 12000.] - 12000.)/12000.

bathymetry = grid.add_zeros('bathymetric__depth', at='node', clobber=True)

gdr = GravityDrivenRouter(grid,
                          diffusivity_cont=2e2/grid.spacing[0],
                          critical_angle_cont=10.,
                          critical_angle_mar=10.,
                          max_erosion_rate_sed=np.inf,
                          max_erosion_rate_br=1e8)

elevations_gdr = np.empty((grid.shape[1], n_iterations//save_step))
for i in tqdm(range(n_iterations)):
    if i%save_step == 0:
        elevations_gdr[:, i//save_step] = elevation.reshape(grid.shape)[1]
    gdr.run_one_step(timestep, update=True)

Now we can compare the two:

In [ ]:
fig, ax = plt.subplots()

# Plot the results from GravityDrivenDiffuser
cmap = mpl.colormaps['Wistia']
colors = cmap(np.linspace(0, 1, elevations_gdd.shape[1]))[:, np.newaxis]
for i in range(elevations_gdd.shape[1]):
    ax.scatter(grid.x_of_node.reshape(grid.shape)[1], elevations_gdd[:, i],
               c=colors[i], s=20, ec='k', lw=0.5, label="GravityDrivenDiffuser")
# Plot the results from WaterDrivenRouter
ax.plot(grid.x_of_node.reshape(grid.shape)[1], elevations_gdr,
        c='k', lw=0.5, label='GravityDrivenRouter')

handles, labels = ax.get_legend_handles_labels()
legend = dict(zip(labels, handles))
ax.legend(legend.values(), legend.keys())
ax.set(xlabel='x (m)', ylabel='z (m)');

Here the critical angles are defined to show a difference with the linear case, not to be plausible. Reducing those angles to 5&#176; lead to unstable results with *GravityDrivenDiffuser* but not with *GravityDrivenRouter*. It is possible to get stable results with *GravityDrivenDiffuser* by reducing the time step. [Carretier et al. (2016)](https://doi.org/10.5194/esurf-4-237-2016) noted that the routing scheme is indeed more stable than the time-explicit finite-volume one.

## 3. Transport-limited model

Now we can move from gravity-driven to water-driven models, starting with a transport-limited model. Again we begin by setting up and running Landlab's component with the same configuration as before:

In [ ]:
grid = RasterModelGrid((3, 49),
                       xy_spacing=(500., 500.),
                       number_of_classes=2,
                       initial_allocation=1,
                       number_of_top_layers=0)
grid.set_closed_boundaries_at_grid_edges(False, True, False, True)

elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation[grid.x_of_node < 12000.] = 1000.*(grid.x_of_node[grid.x_of_node < 12000.])/12000.
elevation[grid.x_of_node >= 12000.] = 1000. - 1000.*(grid.x_of_node[grid.x_of_node >= 12000.] - 12000.)/12000.

water_influx = grid.add_ones('water__unit_flux_in', at='node', clobber=True) # m/yr

fd = FlowDirectorSteepest(grid)
fa = FlowAccumulator(grid, flow_director=fd)

ast = AreaSlopeTransporter(grid,
                          transport_coefficient=8e-5*grid.spacing[0],
                          area_exponent=1.,
                          slope_exponent=1.)

elevations_ast = np.empty((grid.shape[1], n_iterations//save_step))
for i in tqdm(range(n_iterations)):
    if i%save_step == 0:
        elevations_ast[:, i//save_step] = elevation.reshape(grid.shape)[1]
    fa.run_one_step()
    ast.run_one_step(timestep)

Let's visualize our base case:

In [ ]:
fig, ax = plt.subplots()

# Plot the results from Landlab
cmap = mpl.colormaps['Wistia']
colors = cmap(np.linspace(0, 1, elevations_ast.shape[1]))[:, np.newaxis]
for i in range(elevations_ast.shape[1]):
    ax.scatter(grid.x_of_node.reshape(grid.shape)[1], elevations_ast[:, i],
               c=colors[i], s=20, ec='k', lw=0.5)

ax.set(xlabel='x (m)', ylabel='z (m)');

As before, we'll look at two components from StratigraPy that implement different ways of solving the same equations. Let's start with the finite-volume approach:

In [ ]:
grid = RasterModelGrid((3, 49),
                       xy_spacing=(500., 500.),
                       number_of_classes=1,
                       initial_allocation=1,
                       number_of_top_layers=0)
grid.set_closed_boundaries_at_grid_edges(False, True, False, True)

elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation[grid.x_of_node < 12000.] = 1000.*(grid.x_of_node[grid.x_of_node < 12000.])/12000.
elevation[grid.x_of_node >= 12000.] = 1000. - 1000.*(grid.x_of_node[grid.x_of_node >= 12000.] - 12000.)/12000.

water_influx = grid.add_ones('water__unit_flux_in', at='node', clobber=True) # m/yr

bathymetry = grid.add_zeros('bathymetric__depth', at='node', clobber=True)

fd = FlowDirectorSteepest(grid)
fa = FlowAccumulator(grid, flow_director=fd)

wdd = WaterDrivenDiffuser(grid,
                          transportability_cont=8e-5,
                          max_erosion_rate_sed=np.inf,
                          max_erosion_rate_br=1e8)

elevations_wdd = np.empty((grid.shape[1], n_iterations//save_step))
for i in tqdm(range(n_iterations)):
    if i%save_step == 0:
        elevations_wdd[:, i//save_step] = elevation.reshape(grid.shape)[1]
    fa.run_one_step()
    wdd.run_one_step(timestep, update=True)

And compare with Landlab:

In [ ]:
fig, ax = plt.subplots()

# Plot the results from Landlab
cmap = mpl.colormaps['Wistia']
colors = cmap(np.linspace(0, 1, elevations_ast.shape[1]))[:, np.newaxis]
for i in range(elevations_ast.shape[1]):
    ax.scatter(grid.x_of_node.reshape(grid.shape)[1], elevations_ast[:, i],
               c=colors[i], s=20, ec='k', lw=0.5, label='Landlab')
# Plot the results from StratigraPy
ax.plot(grid.x_of_node.reshape(grid.shape)[1], elevations_wdd,
        c='k', lw=0.5, label='WaterDrivenDiffuser')

handles, labels = ax.get_legend_handles_labels()
legend = dict(zip(labels, handles))
ax.legend(legend.values(), legend.keys())
ax.set(xlabel='x (m)', ylabel='z (m)');

Now let's move to the routing approach:

In [ ]:
grid = RasterModelGrid((3, 49),
                       xy_spacing=(500., 500.),
                       number_of_classes=2,
                       initial_allocation=1,
                       number_of_top_layers=0)
grid.set_closed_boundaries_at_grid_edges(False, True, False, True)

elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation[grid.x_of_node < 12000.] = 1000.*(grid.x_of_node[grid.x_of_node < 12000.])/12000.
elevation[grid.x_of_node >= 12000.] = 1000. - 1000.*(grid.x_of_node[grid.x_of_node >= 12000.] - 12000.)/12000.

water_influx = grid.add_ones('water__unit_flux_in', at='node', clobber=True) # m/yr

bathymetry = grid.add_zeros('bathymetric__depth', at='node', clobber=True)

fd = FlowDirectorSteepest(grid)
fa = FlowAccumulator(grid, flow_director=fd)

wdr = WaterDrivenRouter(grid,
                        transportability_cont=8e-5/grid.spacing[0],
                        max_erosion_rate_sed=np.inf,
                        max_erosion_rate_br=1e8)

elevations_wdr = np.empty((grid.shape[1], n_iterations//save_step))
for i in tqdm(range(n_iterations)):
    if i%save_step == 0:
        elevations_wdr[:, i//save_step] = elevation.reshape(grid.shape)[1]
    fa.run_one_step()
    wdr.run_one_step(timestep, update=True)

And compare with Landlab:

In [ ]:
fig, ax = plt.subplots()

# Plot the results from Landlab
cmap = mpl.colormaps['Wistia']
colors = cmap(np.linspace(0, 1, elevations_ast.shape[1]))[:, np.newaxis]
for i in range(elevations_ast.shape[1]):
    ax.scatter(grid.x_of_node.reshape(grid.shape)[1], elevations_ast[:, i],
               c=colors[i], s=20, ec='k', lw=0.5, label='Landlab')
# Plot the results from StratigraPy
ax.plot(grid.x_of_node.reshape(grid.shape)[1], elevations_wdr,
        c='k', lw=0.5, label='WaterDrivenRouter')

handles, labels = ax.get_legend_handles_labels()
legend = dict(zip(labels, handles))
ax.legend(legend.values(), legend.keys())
ax.set(xlabel='x (m)', ylabel='z (m)');

Landlab also uses a routing approach, which looks again more stable than the time-explicit finite-volume approach implemented in *WaterDrivenDiffuser*.

Now let's compare the two components of StratigraPy, using the finite-volume one as base case, except that now we use two sediment classes and a flux limiter by limiting the maximum erosion rate:

In [ ]:
grid = RasterModelGrid((3, 49),
                       xy_spacing=(500., 500.),
                       number_of_classes=2,
                       initial_allocation=1,
                       number_of_top_layers=0)
grid.set_closed_boundaries_at_grid_edges(False, True, False, True)

elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation[grid.x_of_node < 12000.] = 1000.*(grid.x_of_node[grid.x_of_node < 12000.])/12000.
elevation[grid.x_of_node >= 12000.] = 1000. - 1000.*(grid.x_of_node[grid.x_of_node >= 12000.] - 12000.)/12000.

water_influx = grid.add_ones('water__unit_flux_in', at='node', clobber=True) # m/yr

bathymetry = grid.add_zeros('bathymetric__depth', at='node', clobber=True)

fd = FlowDirectorSteepest(grid)
fa = FlowAccumulator(grid, flow_director=fd)

wdd = WaterDrivenDiffuser(grid,
                          transportability_cont=[2e-4, 2e-5],
                          max_erosion_rate_sed=2e-3,
                          max_erosion_rate_br=2e-3)

elevations_wdd = np.empty((grid.shape[1], n_iterations//save_step))
for i in tqdm(range(n_iterations)):
    if i%save_step == 0:
        elevations_wdd[:, i//save_step] = elevation.reshape(grid.shape)[1]
    fa.run_one_step()
    wdd.run_one_step(timestep, update=True)

In [ ]:
fig, ax = plt.subplots()

# Plot the results from WaterDrivenDiffuser
cmap = mpl.colormaps['Wistia']
colors = cmap(np.linspace(0, 1, elevations_wdd.shape[1]))[:, np.newaxis]
for i in range(elevations_wdd.shape[1]):
    ax.scatter(grid.x_of_node.reshape(grid.shape)[1], elevations_wdd[:, i],
               c=colors[i], s=20, ec='k', lw=0.5)

ax.set(xlabel='x (m)', ylabel='z (m)');

Now we can setup and run the routing approach:

In [ ]:
grid = RasterModelGrid((3, 49),
                       xy_spacing=(500., 500.),
                       number_of_classes=2,
                       initial_allocation=1,
                       number_of_top_layers=0)
grid.set_closed_boundaries_at_grid_edges(False, True, False, True)

elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation[grid.x_of_node < 12000.] = 1000.*(grid.x_of_node[grid.x_of_node < 12000.])/12000.
elevation[grid.x_of_node >= 12000.] = 1000. - 1000.*(grid.x_of_node[grid.x_of_node >= 12000.] - 12000.)/12000.

water_influx = grid.add_ones('water__unit_flux_in', at='node', clobber=True) # m/yr

bathymetry = grid.add_zeros('bathymetric__depth', at='node', clobber=True)

fd = FlowDirectorSteepest(grid)
fa = FlowAccumulator(grid, flow_director=fd)

wdr = WaterDrivenRouter(grid,
                        transportability_cont=[2e-4/grid.spacing[0], 2e-5/grid.spacing[0]],
                        max_erosion_rate_sed=2e-3,
                        max_erosion_rate_br=2e-3)

elevations_wdr = np.empty((grid.shape[1], n_iterations//save_step))
for i in tqdm(range(n_iterations)):
    if i%save_step == 0:
        elevations_wdr[:, i//save_step] = elevation.reshape(grid.shape)[1]
    fa.run_one_step()
    wdr.run_one_step(timestep, update=True)

And compare the two:

In [ ]:
fig, ax = plt.subplots()

# Plot the results from WaterDrivenDiffuser
cmap = mpl.colormaps['Wistia']
colors = cmap(np.linspace(0, 1, elevations_wdd.shape[1]))[:, np.newaxis]
for i in range(elevations_wdd.shape[1]):
    ax.scatter(grid.x_of_node.reshape(grid.shape)[1], elevations_wdd[:, i],
               c=colors[i], s=20, ec='k', lw=0.5, label='WaterDrivenDiffuser')
# Plot the results from WaterDrivenRouter
ax.plot(grid.x_of_node.reshape(grid.shape)[1], elevations_wdr,
        c='k', lw=0.5, label='WaterDrivenRouter')

handles, labels = ax.get_legend_handles_labels()
legend = dict(zip(labels, handles))
ax.legend(legend.values(), legend.keys())
ax.set(xlabel='x (m)', ylabel='z (m)');

## 4. SPACE model

While the previous section focused on purely transport-limited models, Landlab implements SPACE, a model that can switch between the transport-limited and the detachment-limited cases. Let's compare Landlab's SPACE component to the equivalent component in StratigraPy, starting with setting up and running Landlab's component:

In [ ]:
grid_spa = RasterModelGrid((3, 49), xy_spacing=(500., 500.))
grid_spa.set_closed_boundaries_at_grid_edges(False, True, False, True)

elevation = grid_spa.add_zeros('topographic__elevation', at='node', clobber=True)
elevation[grid_spa.x_of_node < 12000.] = 1000.*(grid_spa.x_of_node[grid_spa.x_of_node < 12000.])/12000.
elevation[grid_spa.x_of_node >= 12000.] = 1000. - 1000.*(grid_spa.x_of_node[grid_spa.x_of_node >= 12000.] - 12000.)/12000.

water_influx = grid_spa.add_ones('water__unit_flux_in', at='node', clobber=True) # m/yr

soil_depth = grid_spa.add_zeros('soil__depth', at='node', clobber=True) # m

fd = FlowDirectorSteepest(grid_spa)
fa = FlowAccumulator(grid_spa, flow_director=fd)

spa = SpaceLargeScaleEroder(grid_spa,
                            K_sed=8e-5,
                            K_br=8e-5,
                            phi=0.3,
                            H_star=0.1,
                            v_s=1.,
                            m_sp=0.5,
                            n_sp=1.)

elevations_spa = np.empty((grid_spa.shape[1], n_iterations//save_step))
for i in tqdm(range(n_iterations)):
    if i%save_step == 0:
        elevations_spa[:, i//save_step] = elevation.reshape(grid_spa.shape)[1]
    fa.run_one_step()
    spa.run_one_step(timestep)

In [ ]:
fig, ax = plt.subplots()

# Plot the results from Landlab
cmap = mpl.colormaps['Wistia']
colors = cmap(np.linspace(0, 1, elevations_spa.shape[1]))[:, np.newaxis]
for i in range(elevations_spa.shape[1]):
    ax.scatter(grid_spa.x_of_node.reshape(grid_spa.shape)[1], elevations_spa[:, i],
               c=colors[i], s=20, ec='k', lw=0.5)

ax.set(xlabel='x (m)', ylabel='z (m)');

And do the same for StratigraPy's *FluxDrivenRouter*:

In [ ]:
grid = RasterModelGrid((3, 49),
                       xy_spacing=(500., 500.),
                       number_of_classes=1,
                       initial_allocation=1,
                       number_of_top_layers=0)
grid.set_closed_boundaries_at_grid_edges(False, True, False, True)

elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation[grid.x_of_node < 12000.] = 1000.*(grid.x_of_node[grid.x_of_node < 12000.])/12000.
elevation[grid.x_of_node >= 12000.] = 1000. - 1000.*(grid.x_of_node[grid.x_of_node >= 12000.] - 12000.)/12000.

water_influx = grid.add_ones('water__unit_flux_in', at='node', clobber=True) # m/yr

bathymetry = grid.add_zeros('bathymetric__depth', at='node', clobber=True)

fd = FlowDirectorSteepest(grid)
fa = FlowAccumulator(grid, flow_director=fd)

fdr = FluxDrivenRouter(grid,
                       erodibility_sed_cont=8e-5,
                       settling_velocity=1.,
                       critical_thickness=0.1,
                       porosity=0.3,
                       max_erosion_rate_sed=np.inf,
                       erodibility_br_cont=8e-5,
                       exponent_discharge=0.5,
                       exponent_slope=1.)

elevations_fdr = np.empty((grid.shape[1], n_iterations//save_step))
for i in tqdm(range(n_iterations)):
    if i%save_step == 0:
        elevations_fdr[:, i//save_step] = elevation.reshape(grid.shape)[1]
    fa.run_one_step()
    fdr.run_one_step(timestep, update=True)

Now we can compare the results:

In [ ]:
fig, ax = plt.subplots()

# Plot the results from Landlab
cmap = mpl.colormaps['Wistia']
colors = cmap(np.linspace(0, 1, elevations_spa.shape[1]))[:, np.newaxis]
for i in range(elevations_spa.shape[1]):
    ax.scatter(grid.x_of_node.reshape(grid.shape)[1], elevations_spa[:, i],
               c=colors[i], s=20, ec='k', lw=0.5, label='Landlab')
# Plot the results from StratigraPy
ax.plot(grid.x_of_node.reshape(grid.shape)[1], elevations_fdr,
        c='k', lw=0.5, label='FluxDrivenRouter')

handles, labels = ax.get_legend_handles_labels()
legend = dict(zip(labels, handles))
ax.legend(legend.values(), legend.keys())
ax.set(xlabel='x (m)', ylabel='z (m)');

The results are not quite as similar as they seem though. Checking the sediments deposited by both models, we can see that *SpaceLargeScaleEroder* deposits some:

In [ ]:
grid_spa.at_node['soil__depth'].reshape(grid_spa.shape)[1]

while *FluxDrivenRouter* doesn't:

In [ ]:
grid.stacked_layers.thickness

SPACE computes erosion and deposition, with some deposition always happening, except in the pure detachment-limited case. StratigraPy on the other end doesn't implement the full SPACE model. Its router scheme just updates sediments based on the difference between sediment influx and outflux in a cell, which means that if erosion dominates, no deposition will happen. So StratigraPy underestimates deposition compared to SPACE.